## Setup
1. Run YAP API server
1. `pip install -r requirements.txt`
1. Run NEMO API server `uvicorn api_main:app --port 8090`
1. Have a look at the swagger OpenAPI documentation by opening http://localhost:8090/docs in your browser

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

In [3]:
requests.get('http://localhost:8090/').json()

{'message': 'Please specify command in URL path in a POST request and provide some input text in the request body.',
 'available_commands': ['run_ncrf_model',
  'multi_align_hybrid',
  'multi_to_single',
  'morph_yap',
  'morph_hybrid',
  'morph_hybrid_align_tokens']}

In [4]:
texts = [
    'מלך השערים במונדיאל 2006 היה מירוסלב קלוזה.',
    "עשרות אנשים מגיעים מתאילנד לישראל.\nתופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת."
]

## Calling the API
1. All endpoints are HTTP POST
1. Request body contains a json with Hebrew `sentences` and optional `tokenized` flag for signaling whether they are tokenized or not.
1. Request URL may include further optional parameters for choosing models/scenarios (in all but `run_ner_model` there is no need to touch these)
1. Response is in the response body as a JSON 

## Run NER Model

In [5]:
headers =   {   'accept': 'application/json', 
                'Content-Type': 'application/json' }

In [6]:
%%time 
params = {
            'model_name': 'token-multi', # / 'token-single' / 'morph' 
}
payload = { 'sentences': texts[1],
            #'tokenized': True,
          }

res = requests.post('http://localhost:8090/run_ncrf_model', params=params, json=payload, headers=headers).json()
list(zip(res[0]['tokenized_text'], res[0]['ncrf_preds'])), list(zip(res[1]['tokenized_text'], res[1]['ncrf_preds']))

CPU times: user 3.58 ms, sys: 1.88 ms, total: 5.46 ms
Wall time: 30.8 ms


([('עשרות', 'O'),
  ('אנשים', 'O'),
  ('מגיעים', 'O'),
  ('מתאילנד', 'O^S-GPE'),
  ('לישראל', 'O^S-GPE'),
  ('.', 'O')],
 [('תופעה', 'O'),
  ('זו', 'O'),
  ('התבררה', 'O'),
  ('אתמול', 'O'),
  ('בוועדת', 'O^B-ORG'),
  ('העבודה', 'I-ORG^I-ORG'),
  ('והרווחה', 'I-ORG^I-ORG^E-ORG'),
  ('של', 'O'),
  ('הכנסת', 'B-ORG^E-ORG'),
  ('.', 'O')])

### multi_to_single (get token-single predictions using a token-multi model)

In [7]:
%%time
params = {
            #'multi_model_name': 'token-multi',

}
payload = { 'sentences': texts[1],
            #'tokenized': True,
          }

res = requests.post('http://localhost:8090/multi_to_single', params=params, json=payload, headers=headers).json()
res

CPU times: user 2.17 ms, sys: 2.51 ms, total: 4.67 ms
Wall time: 28.2 ms


[{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
  'ents': {'token': {'nemo_multi_align_token': [{'text': 'מתאילנד',
      'label': 'GPE',
      'start': 3,
      'end': 4},
     {'text': 'לישראל', 'label': 'GPE', 'start': 4, 'end': 5}]}},
  'tokens': [{'text': 'עשרות', 'nemo_multi_align_token': 'O'},
   {'text': 'אנשים', 'nemo_multi_align_token': 'O'},
   {'text': 'מגיעים', 'nemo_multi_align_token': 'O'},
   {'text': 'מתאילנד', 'nemo_multi_align_token': 'S-GPE'},
   {'text': 'לישראל', 'nemo_multi_align_token': 'S-GPE'},
   {'text': '.', 'nemo_multi_align_token': 'O'}]},
 {'text': 'תופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת.',
  'ents': {'token': {'nemo_multi_align_token': [{'text': 'בוועדת העבודה והרווחה',
      'label': 'ORG',
      'start': 4,
      'end': 7},
     {'text': 'הכנסת', 'label': 'ORG', 'start': 8, 'end': 9}]}},
  'tokens': [{'text': 'תופעה', 'nemo_multi_align_token': 'O'},
   {'text': 'זו', 'nemo_multi_align_token': 'O'},
   {'text': 'התבררה', 'nemo_multi_align

## MD + NER

### morph_yap

In [8]:
%%time
params = {
            #'morph_model_name': 'morph',
}
payload = { 'sentences': texts[1],
            #'tokenized': True,
          }

res = requests.post('http://localhost:8090/morph_yap', params=params, json=payload, headers=headers).json()
res[0]

CPU times: user 2.91 ms, sys: 2.69 ms, total: 5.6 ms
Wall time: 176 ms


{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
 'ents': {'morph': {'nemo_morph': [{'text': 'תאילנד',
     'label': 'GPE',
     'start': 4,
     'end': 5},
    {'text': 'ישראל', 'label': 'GPE', 'start': 6, 'end': 7}]}},
 'tokens': [{'text': 'עשרות',
   'morphs': [{'form': 'עשרות',
     'nemo_morph': 'O',
     'lemma': 'עשר',
     'pos': 'CDT',
     'feats': 'gen=F|num=P'}]},
  {'text': 'אנשים',
   'morphs': [{'form': 'אנשים',
     'nemo_morph': 'O',
     'lemma': 'איש',
     'pos': 'NN',
     'feats': 'gen=M|num=P'}]},
  {'text': 'מגיעים',
   'morphs': [{'form': 'מגיעים',
     'nemo_morph': 'O',
     'lemma': 'הגיע',
     'pos': 'BN',
     'feats': 'gen=M|num=P|per=A'}]},
  {'text': 'מתאילנד',
   'morphs': [{'form': 'מ',
     'nemo_morph': 'O',
     'lemma': 'מ',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'תאילנד',
     'nemo_morph': 'S-GPE',
     'lemma': 'תאילנד',
     'pos': 'NNP',
     'feats': 'gen=F|num=S'}]},
  {'text': 'לישראל',
   'morphs': [{'form': 'ל',
     

### multi_align_hybrid

In [9]:
%%time
payload = { 'sentences': texts[1],
            #'tokenized': True,
          }

res = requests.post('http://localhost:8090/multi_align_hybrid', params=params, json=payload, headers=headers).json()
res[0]

CPU times: user 5.38 ms, sys: 793 µs, total: 6.17 ms
Wall time: 122 ms


{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
 'ents': {'morph': {'nemo_multi_align_morph': [{'text': 'תאילנד',
     'label': 'GPE',
     'start': 4,
     'end': 5},
    {'text': 'ישראל', 'label': 'GPE', 'start': 6, 'end': 7}]}},
 'tokens': [{'text': 'עשרות',
   'morphs': [{'form': 'עשרות',
     'nemo_multi_align_morph': 'O',
     'lemma': 'עשר',
     'pos': 'CDT',
     'feats': 'gen=F|num=P'}]},
  {'text': 'אנשים',
   'morphs': [{'form': 'אנשים',
     'nemo_multi_align_morph': 'O',
     'lemma': 'איש',
     'pos': 'NN',
     'feats': 'gen=M|num=P'}]},
  {'text': 'מגיעים',
   'morphs': [{'form': 'מגיעים',
     'nemo_multi_align_morph': 'O',
     'lemma': 'הגיע',
     'pos': 'BN',
     'feats': 'gen=M|num=P|per=A'}]},
  {'text': 'מתאילנד',
   'morphs': [{'form': 'מ',
     'nemo_multi_align_morph': 'O',
     'lemma': 'מ',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'תאילנד',
     'nemo_multi_align_morph': 'S-GPE',
     'lemma': 'תאילנד',
     'pos': 'NNP',
     'feats': 

### morph_hybrid
The following models got the best results in our experiments.


In [10]:
%%time
params = {
            #'multi_model_name': 'token-multi',
            #'morph_model_name': 'morph',
            # 'align_tokens': False,
}
payload = { 'sentences': texts[1],
            #'tokenized': False,
          }

res = requests.post('http://localhost:8090/morph_hybrid', params=params, json=payload, headers=headers).json()
res[0]

CPU times: user 4.49 ms, sys: 1.5 ms, total: 5.99 ms
Wall time: 117 ms


{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
 'ents': {'morph': {'nemo_morph': [{'text': 'תאילנד',
     'label': 'GPE',
     'start': 4,
     'end': 5},
    {'text': 'ישראל', 'label': 'GPE', 'start': 6, 'end': 7}]}},
 'tokens': [{'text': 'עשרות',
   'morphs': [{'form': 'עשרות',
     'nemo_morph': 'O',
     'lemma': 'עשר',
     'pos': 'CDT',
     'feats': 'gen=F|num=P'}]},
  {'text': 'אנשים',
   'morphs': [{'form': 'אנשים',
     'nemo_morph': 'O',
     'lemma': 'איש',
     'pos': 'NN',
     'feats': 'gen=M|num=P'}]},
  {'text': 'מגיעים',
   'morphs': [{'form': 'מגיעים',
     'nemo_morph': 'O',
     'lemma': 'הגיע',
     'pos': 'BN',
     'feats': 'gen=M|num=P|per=A'}]},
  {'text': 'מתאילנד',
   'morphs': [{'form': 'מ',
     'nemo_morph': 'O',
     'lemma': 'מ',
     'pos': 'PREPOSITION',
     'feats': '_'},
    {'form': 'תאילנד',
     'nemo_morph': 'S-GPE',
     'lemma': 'תאילנד',
     'pos': 'NNP',
     'feats': 'gen=F|num=S'}]},
  {'text': 'לישראל',
   'morphs': [{'form': 'ל',
     

### morph_hybrid_align_tokens

In [11]:
%%time
params = {
            'verbose': 1,
            'include_yap_outputs': True,
}
payload = { 'sentences': texts[1],
            #'tokenized': False,
          }

res = requests.post('http://localhost:8090/morph_hybrid_align_tokens', params=params, json=payload, headers=headers).json()
res[0]

CPU times: user 6.44 ms, sys: 0 ns, total: 6.44 ms
Wall time: 167 ms


{'text': 'עשרות אנשים מגיעים מתאילנד לישראל.',
 'ents': {'morph': {'nemo_morph': [{'text': 'תאילנד',
     'label': 'GPE',
     'start': 4,
     'end': 5},
    {'text': 'ישראל', 'label': 'GPE', 'start': 6, 'end': 7}],
   'nemo_multi_align_morph': [{'text': 'תאילנד',
     'label': 'GPE',
     'start': 4,
     'end': 5},
    {'text': 'ישראל', 'label': 'GPE', 'start': 6, 'end': 7}]},
  'token': {'nemo_multi_align_token': [{'text': 'מתאילנד',
     'label': 'GPE',
     'start': 3,
     'end': 4},
    {'text': 'לישראל', 'label': 'GPE', 'start': 4, 'end': 5}],
   'nemo_morph_align_token': [{'text': 'מתאילנד',
     'label': 'GPE',
     'start': 3,
     'end': 4},
    {'text': 'לישראל', 'label': 'GPE', 'start': 4, 'end': 5}]}},
 'tokens': [{'text': 'עשרות',
   'nemo_multi': 'O',
   'nemo_multi_align_token': 'O',
   'nemo_morph_align_token': 'O',
   'morphs': [{'form': 'עשרות',
     'nemo_morph': 'O',
     'nemo_multi_align_morph': 'O',
     'lemma': 'עשר',
     'pos': 'CDT',
     'feats': 'gen=F

## Display and view ents


In [22]:
from notebook_utils import *
e = EntityRenderer()

In [66]:
payload = { 'sentences': '\n'.join(texts+['גנן גידל דגן בגן']),}
params = {
            'verbose': 0,
            #'align_tokens': False,
}
res = requests.post('http://localhost:8090/morph_hybrid', params=params, json=payload, headers=headers).json()

In [67]:
res[-1]['ents']

{'morph': {'nemo_morph': [{'text': 'גן',
    'label': 'LOC',
    'start': 4,
    'end': 5}]}}

In [68]:
from IPython.core.display import display, HTML
display(HTML(e.render(res, 'morph', 'nemo_morph')))


In [69]:
ents = ents_to_df(res)
ents

,sent_id,text,label,level,scenario
0,0,מירוסלב קלוזה,PER,morph,nemo_morph
1,1,תאילנד,GPE,morph,nemo_morph
2,1,ישראל,GPE,morph,nemo_morph
3,2,וועדת ה עבודה ו ה רווחה,ORG,morph,nemo_morph
4,2,ה כנסת,ORG,morph,nemo_morph
5,3,גן,LOC,morph,nemo_morph
